First, we import the necessary modules

In [ ]:
from dtr import DTR
import pandas as pd
from keras.models import load_model

Then we load the data, and we clearly specify the inputs and the outputs 

In [ ]:
data = pd.read_csv('./artificial_data.csv',index_col=0, infer_datetime_format=True, parse_dates=['datetime'])
inputs = ['Wind Speed [m/s]', 'Arranged Wind Dir [°]', 'Air temp [°C]', 'Humidity [%]', 'Sun irradiance thermal flow absorbed by the conductor.[W/m]', 'Current flow [A]']
output = ['Actual Conductor Temp (t+1) [°C]']

We instantiate our DTR object as follows, passing the `data`, `inputs`, and `outputs` parameter from before. The DTR class provides user-friendly functions to easily perform the proposed experiments and adapt them to run with your own data. 

In [ ]:
dtr = DTR(data, inputs, output)

The `compute_sensors_distance` function computes various statistics that measure the distance between pairs of sensors in the given dataset. Specifically, it computes the mean difference, standard deviation difference, euclidean distance, manhattan distance, cosine similarity, and KDE area difference between the two sensor dataframes. 

In [ ]:
dtr.compute_sensors_distance()

If the dataset are particularly large, the execution might be long. Therefore, it is recommended to export the computed distances after the first execution, and then load them for the following executions, as follows: 

In [ ]:
dtr.export_distances('./distances.csv')
dtr.load_distances('./distances.csv')

You can inspect the metrics measured as follows, and find the couple of sensors that minimizes the distance of your preference. 

In [ ]:
print(dtr.get_distances())

Before starting, we will load our base regressor.


In [ ]:
model = load_model('model.h5')

And here is the core of our code. The `transfer` function is designed to perform transfer learning from one sensor (identified by the `source_sensor_id` parameter) to another sensor (identified by the `target_sensor_id parameter`) using a specified method. 

In [ ]:
dtr.transfer(source_sensor_id = 7, 
            target_sensor_id= 1, 
            method = 'parameter_based_transfer', 
            target_num_available_days = 15, 
            target_num_test_days=30, 
            sliding_window = False, 
            regressor=model, 
            verbose = 1, 
            epochs = 5,  
            batch_size = 500, 
            ieee=True, 
            estimators_tradaboost = 5,
            store_predictions=True)     

The `method` parameter can take on the value `parameter_based_transfer`, `instance_based_transfer`, `both`. The differences between the methods are explained in the [Methods](#methods) section.  which suggests that the function performs some form of parameter-based transfer learning.

The `target_num_available_days` specifies the number of days from the target sensor that are available when performing transfer learning. `target_num_test_days` instead indicates the number of days to be used as a test set. Exactly which days are considered is chosen as follows: for the test set it's always the last `target_num_test_days` of the target sensor; for the available data there are two options. If `sliding_window` is False, a random window of `target_num_available_days` is chosen between the beginning of the target sensor data and the beginning of the test set.  If `sliding_window` is True, *all possible windows* of size `target_num_available_days` that are located between the beginning of the target sensor data and the beginning of the test set are tested, and the result is averaged. (Notice that this might be particularly slow!)

The `regressor` parameter specifies a model that will be used to perform the transfer. The `epochs` and `batch_size` parameters specify the number of epochs and batch size to be used in training the model, when the model is a Neural Network.  The `estimators_tradaboost` parameter specifies the number of estimators to use in the TrAdaBoost.R2 instance-based approach. 

The `ieee` parameter determines whether or not the function should consider the IEEE738 estimation for the conductor temperature in the methods comparison. NB: the function `transfer()` **does not compute the IEEE738 estimation** for the conductor temperature ! It should be present in the dataset when setting the `ieee` parameter to `True`. 

The `store_predictions` parameter determines whether or not to keep the predictions made by the model. This defaults to False, because storing all the predictions might significantly increase RAM usage. 

Finally, the `verbose` parameter controls the level of output produced by the function.

| Parameter                 | Default | Meaning                                                                                                     |
|---------------------------|:-------:|-------------------------------------------------------------------------------------------------------------|
| source_sensor_id          |   N.A.  | ID of the sensor from which data is transferred                                                             |
| target_sensor_id          |   N.A.  | ID of the sensor to which data is transferred                                                               |
| method                    |  'both' | Method of transfer learning to use, either 'parameter_based_transfer', 'instance_based_transfer', or 'both' |
| target_num_available_days |    15   | Number of days from the target sensor to use for training the model                                         |
| target_num_test_days      |    30   | Number of days from the target sensor to use as a test set                                                  |
| sliding_window            |  False  | Whether to use all possible windows of size target_num_available_days or a random one                       |
| regressor                 |  model  | Base Model to use for transfer learning: must be a Neural Network for 'parameter_based_transfer' or 'both'  |
| verbose                   |    0    | Level of output produced by the function                                                                    |
| epochs                    |    15   | Number of epochs to use when training the model (if the model is a neural network)                          |
| batch_size                |    32   | Batch size to use when training the model (if the model is a neural network)                                |
| ieee                      |  False  | Whether to consider the IEEE738 estimation for the conductor temperature in the methods comparison          |
| estimators_tradaboost     |    20   | Number of estimators to use in the TrAdaBoost.R2 instance-based approach                                    |
| store_predictions         |   True  | Whether to keep the predictions made by the model                                                           |

The method `plot_results()` can be used to plot the columns of the results obtained from the previous. The plot has the testing day on the x-axis and the metric value on the y-axis, for all the different teted methods. 

In [ ]:
dtr.plot_results()

If the user needs the results table, for specific manipulations, she can get it using the following method `get_results()`. 

In [ ]:
results = dtr.get_results()

If the `store_predictions` was `True` in the `dtr.transfer()` function, the user can decide to visualize the temperature predicted by each method on one specific testing day.  the used needs the results table, for specific manipulations, she can get it using the following method `get_results`. 

In [ ]:
dtr.plot_predictions_for_day('2022-01-28')

In [ ]:
dtr.get_predictions()